**1.Import Libraries**

In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

**2.Load Training Dataset**

In [7]:
dataset_train=pd.read_csv("stock.csv")
dataset_train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-25,7.232500,7.310714,7.149643,7.252500,6.131644,1065699600
1,2010-01-26,7.355357,7.632500,7.235000,7.355000,6.218304,1867110000
2,2010-01-27,7.387500,7.520714,7.126071,7.424286,6.276882,1722568400
3,2010-01-28,7.318929,7.339286,7.096429,7.117500,6.017509,1173502400
4,2010-01-29,7.181429,7.221429,6.794643,6.859286,5.799202,1245952400


**3.Use Open Stock Price Column to train your model**

In [19]:
training_set=dataset_train.iloc[:,1:2].values
print(training_set)
print(training_set.shape)

[[  7.2325  ]
 [  7.355357]
 [  7.3875  ]
 ...
 [224.369995]
 [224.      ]
 [218.869995]]
(3650, 1)


**4.Normalizing the dataset**

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler(feature_range=(0,1))
scaled_training_set=scaler.fit_transform(training_set)
scaled_training_set

array([[0.00157721],
       [0.00211228],
       [0.00225227],
       ...,
       [0.94725831],
       [0.9456469 ],
       [0.92330461]])

**5.Creating x_train and y_train data structures**

In [24]:
X_train=[]
Y_train=[]
for i in range(60,1258):
    X_train.append(scaled_training_set[i-60:i,0])
    Y_train.append(scaled_training_set[i,0])
X_train=np.array(X_train)
Y_train=np.array(Y_train)
print (X_train.shape)
print(Y_train.shape)

(1198, 60)
(1198,)


**6.Reshape the data**

In [28]:
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_train.shape

(1198, 60, 1)

**7.Building MOdel by importimg crucial libraries and adding diff layes to oLSTM**

In [33]:
from keras.models import Sequential 
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

ModuleNotFoundError: No module named 'tensorflow.python'